In [ ]:
import sys
sys.path.append('../../PlateTectonicTools/')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt
import seaborn as sns

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.geometry import nearest_feature

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paleoelevation//python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap
from time_series import lowess

import collections
import matplotlib.ticker as mticker

%matplotlib inline
%load_ext autoreload
%autoreload 2

model_dir = '../../andes_paleoelevation//luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)



In [ ]:
def my_boxplot(df_binned, ax, colname,
               color='black', widths=50):
    
    midpoints = [key for key in list(df_binned.groups.keys())]

    median_elevations = []
    elevation_violins = []
    for group in df_binned:
        if not group[1].empty:
            median_elevations.append(group[1][colname].median())
            elevation_violins.append(group[1][colname].dropna())
            #print(elevation_violins)
            #ax.plot(group[0], elevation_violins)
        else:
            median_elevations.append(np.nan)
            elevation_violins.append([np.nan,np.nan])

    parts = ax.boxplot(elevation_violins, positions=midpoints, widths=widths, 
                       patch_artist=True, manage_ticks=False, showfliers=False)
    for pc in parts['boxes']:
        pc.set_facecolor(color)
        pc.set_alpha(0.5)
        pc.set_edgecolor(color)
        

In [ ]:
M2021 = Reconstructions.fetch_Merdith2021()
TectonicMap = Geology.fetch_GlobalTectonicMap()

TectonicMap = TectonicMap.rename({'Zealandia': 'Australia'})

df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')

df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']), crs='EPSG:4326')

sz_distance_limit = 750.
df = df.query('distance_to_sz<@sz_distance_limit').reset_index(drop=True)

time_bin_size = 25


#df['bin_age'] = np.round(df['age']/time_bin_size) * time_bin_size
df['bin_age'] = joy.assign_bin_ages(df['age'], time_bin_size)

#df = M2021.assign_plate_ids(df, copy_valid_times=True)

#df = M2021.reconstruct_to_time_of_appearance(df, ReconstructTime='bin_age')

df.plot()


In [ ]:
#calibration = 'luffi'
#mohometer_selection = 41
#mohometer_selection = ['la_yb_elevation']

space_bin_size = 2
sz_distance_limit = 750.

sample_medians = True

plot_calibrations = [
    ('luffi', 42),
    #('luffi', 'la_yb_elevation'),
    #('Hu', 'la_yb_elevation'),
    #('Hu', 'sr_y_elevation'),
    #('FarnerLee', 'la_yb_elevation'),
    #('FarnerLee', 'gd_yb_elevation')
]


for space_bin_size in [1,2,5]:
    for calibration, mohometer_selection in plot_calibrations:


        if isinstance(mohometer_selection, list):
            mohometer_description_string = '|'.join(mohometer_selection)
        else:
            mohometer_description_string = str(mohometer_selection)


        df_sel = M2021.reconstruct_to_time_of_appearance(df, ReconstructTime='bin_age')

        df_filt = joy.filter_the_database(df_sel, 
                                          filter_method=calibration,
                                          #age_min=0.1, 
                                          age_max=1000.,
                                          nans_to_zeros=False) 

        df_sel = df_filt.query('distance_to_sz<@sz_distance_limit').reset_index(drop=True)


        elevation_estimates = joy.get_elevations(df_sel, 
                                                 gc_interpolator_dict=gc_interpolator_dict,
                                                 calibration=calibration,
                                                 mohometer_selection=mohometer_selection)
        elevation_estimates = gpd.GeoDataFrame(pd.DataFrame(elevation_estimates).join(df_sel['bin_age']),
                                               geometry=df_sel.geometry)

        #elevation_estimates['median_elevations'] = elevation_estimates.drop(
        #    columns=['bin_age', 'geometry']).median(axis=1)
        #elevation_estimates['moho'] = 6.79 * (elevation_estimates['median_elevations']/1000.) + 26.4
        #elevation_estimates['thickness'] = elevation_estimates['moho'] + elevation_estimates['median_elevations']/1000.

        elevation_estimates['bin_latitude'] = np.round(elevation_estimates.geometry.y/space_bin_size) * space_bin_size
        elevation_estimates['bin_longitude'] = np.round(elevation_estimates.geometry.x/space_bin_size) * space_bin_size

        age_groups = elevation_estimates.groupby(by=['bin_age'])
        age_keys = []
        median_elevations = []
        elevation_sems = []
        nbins_list = []
        elevation_violins = []

        for name, group in age_groups:
            age_keys.append(name[0])
            geog_groups = group.groupby(by=['bin_latitude', 'bin_longitude'])

            '''
            tmp = pd.DataFrame(data={'x':geog_groups.bin_longitude.median().tolist(),
                                     'y':geog_groups.bin_latitude.median().tolist(),
                                     'z':geog_groups.la_yb_elevation.median().tolist()}).dropna(subset='z')
            fig=pygmt.Figure()
            fig.basemap(frame='afg', projection='N15c', region='d')
            M2021.polygon_snapshot('continents', name+0.001).plot(fig, fill='grey', pen='0p,gray')
            M2021.plate_snapshot(name).plot_boundaries(fig)
            pygmt.makecpt(cmap='cubhelix', series=[-1000,5000,500], background=True, reverse=True)

            #fig.plot(x=geog_groups.bin_longitude.median(), y=geog_groups.bin_latitude.median(), fill=geog_groups.la_yb_elevation.median(), 
            #         style='c0.1c', pen='k', cmap=True)
            if len(tmp)>0:
                fig.plot(x=tmp.x, y=tmp.y, fill=tmp.z, 
                         style='c0.2c', pen='0.2p,black', cmap=True)
            fig.savefig('../images/sequence_M2021/time_series_bins_{:0.0f}Ma.png'.format(name))
            #fig.show()
            '''

            binned_list = []
            for n,g in geog_groups:
                data = g.drop(columns=['bin_age', 
                                       'geometry', 
                                       'bin_latitude', 
                                       'bin_longitude'])

                #print(data.median(axis=1))
                #print(data.stack())
                #break

                #print(data.median(axis=1).quantile(q=0.25))
                #print(data.stack().quantile(q=0.25))

                if sample_medians:
                    data = data.median(axis=1).dropna()
                else:
                    data = data.stack().dropna()
                    #print(data.shape)

                binned_list.append([n[1], 
                                    n[0], 
                                    name,
                                    data.median(),
                                    data.quantile(q=0.25),
                                    data.quantile(q=0.75)])

            binned_df = pd.DataFrame(data=binned_list, 
                                     columns=['x', 'y', 'bin_age',
                                              'median_elevation', 
                                              'q25', 'q75'])

            elevation_violins.append(np.array(binned_df.median_elevation.dropna()))
            median_elevations.append(binned_df.median_elevation.median())
            elevation_sems.append(binned_df.median_elevation.sem())
            nbins_list.append(len(np.array(binned_df.median_elevation.dropna())))
            #print(name,
            #      binned_df.median_elevation.median(),
            #      binned_df.median_elevation.sem())
                  #len(tmp))
            #break


        color='black'

        fig = plt.figure(figsize=(7,6))
        gs = fig.add_gridspec(nrows=2, wspace=0.12, hspace=0.0, height_ratios=[1.5,5])
        ax = fig.add_subplot(gs[1, 0])
        #print(age_keys)
        parts = ax.boxplot(elevation_violins, positions=age_keys, widths=time_bin_size*0.8, 
                           patch_artist=True, manage_ticks=False, showfliers=False)
        for pc in parts['boxes']:
            pc.set_facecolor(color)
            pc.set_alpha(0.5)
            pc.set_edgecolor(color)
        ax.set_ylim(-500,4500)
        ax.set_xlim(0,1000)
        ax.set_xlabel('Reconstruction Age [Ma]')
        ax.set_ylabel('Elevation [m]')
        ax.grid()

        ax = fig.add_subplot(gs[0, 0])
        ax.bar(age_keys, nbins_list, width=time_bin_size, 
                color='gray')
        ax.set_xticks([])
        ax.set_xlim(0,1000)
        #ax.set_ylim(1,20000)
        ax.set_yscale('log')
        ax.grid(which='both', axis='y', linestyle='--')


        plt.savefig(
            '../images/boxplots/thickness_time_series_{:s}_{:s}_timebin_{:0.0f}_spacebin_{:0.1f}.png'.format(
                calibration,
                mohometer_description_string,
                time_bin_size,
                space_bin_size), 
            dpi=600)
        plt.close()
        #plt.show()
        #break

    

In [ ]:
#plt.plot(elevation_estimates['bin_age'], elevation_estimates['la_yb_elevation'], 'o')


In [ ]:
continents = ['North America', 'Europe', 'Asia', 
              'South America', 'Antarctica', 'Australia']

plot_calibrations = [
    ('luffi', 41),
    #('luffi', 'la_yb_elevation'),
    #('Hu', 'la_yb_elevation'),
    #('Hu', 'sr_y_elevation'),
    #('FarnerLee', 'la_yb_elevation'),
    #('FarnerLee', 'gd_yb_elevation')
]

ncols = 3

for calibration, mohometer_selection in plot_calibrations:

    if isinstance(mohometer_selection, list):
        mohometer_description_string = '|'.join(mohometer_selection)
    else:
        mohometer_description_string = str(mohometer_selection)

    #'''
    fig = plt.figure(figsize=(16,8))

    gs = fig.add_gridspec(nrows=6, ncols=ncols, wspace=0.12, hspace=0.0, height_ratios=[2,2,5,0.8,2,5])


    for i,continent in enumerate(continents):

        df_sel = joymap.select_orogens(df,
                                   gdf=TectonicMap, 
                                   continent_names=continent)

        df_sel = M2021.reconstruct_to_time_of_appearance(df_sel, ReconstructTime='bin_age')

        df_filt = joy.filter_the_database(df_sel, 
                                      filter_method=calibration,
                                      age_min=0.1, 
                                      age_max=1000.) 

        df_sel = df_filt.query('distance_to_sz<@sz_distance_limit').reset_index(drop=True)

        elevation_estimates = joy.get_elevations(df_sel, 
                                                 gc_interpolator_dict=gc_interpolator_dict,
                                                 calibration=calibration,
                                                 mohometer_selection=mohometer_selection)
        elevation_estimates = gpd.GeoDataFrame(pd.DataFrame(elevation_estimates).join(df_sel['bin_age']),
                                               geometry=df_sel.geometry)

        elevation_estimates['median_elevations'] = elevation_estimates.drop(
            columns=['bin_age', 'geometry']).median(axis=1)

        elevation_estimates['moho'] = 6.79 * (elevation_estimates['median_elevations']/1000.) + 26.4
        elevation_estimates['thickness'] = elevation_estimates['moho'] + elevation_estimates['median_elevations']/1000.


        if i>ncols-1:
            row=5
        else:
            row=2
        ax = fig.add_subplot(gs[row, np.mod(i,ncols)])

        my_boxplot(elevation_estimates.reset_index().groupby(by='bin_age'), ax, 'thickness', widths=time_bin_size*0.8)
        ##ax.plot(elevation_estimates['bin_age'], elevation_estimates['median_elevations'], 'o')
        #ax.set_title(continent)
        #ax.set_xlim(0,1000)
        #ax.set_ylim(25,90)
        #ax.grid()

        ax.set_xlim(0,1000)
        ax.set_ylim(25,90)
        ax.grid()
        if row==5:
            ax.set_xlabel('Age [Ma]')
        else:
            ax.set_xticklabels([])
        if np.mod(i,ncols)==0:
            ax.set_ylabel('Thickness [km]')
        elif np.mod(i,ncols)==ncols-1:
            ax.yaxis.tick_right()
            ax.yaxis.set_label_position("right")
            ax.set_ylabel('Elevation [km]')
            ticks_loc = ax.get_yticks().tolist()
            ax.yaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
            ax.set_yticklabels(['{:0.1f}'.format(t/1000.) for t in joy.thickness2elevation(1000*ax.get_yticks())])
        else:
            ax.set_yticklabels([])

        ax = fig.add_subplot(gs[row-1, np.mod(i,ncols)])
        ax.hist(elevation_estimates.bin_age, 
                bins=np.arange(-time_bin_size/2,1001,time_bin_size),
                color='gray')
        ax.set_xticks([])
        ax.set_xlim(0,1000)
        ax.set_ylim(1,20000)
        ax.set_yscale('log')
        ax.grid(which='both', axis='y', linestyle='--')
        ax.set_title('{:s} '.format(continent), loc='right', y=1.0, pad=-15)
        if np.mod(i,ncols)!=0:
            ax.set_yticklabels([])

    plt.savefig(
        '../images/boxplots/thickness_time_series_by_continent_{:s}_{:s}_timebin_{:0.0f}.png'.format(
            calibration,
            mohometer_description_string,
            time_bin_size), 
        dpi=600,
        bbox_inches='tight')
    #plt.close()
    plt.show()
    #'''  
    